# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tralee,52.2704,-9.7026,13.65,93,98,2.77,IE,1692931791
1,1,bilibino,68.0546,166.4372,12.66,41,87,0.96,RU,1692931792
2,2,ormara,25.2088,64.6357,26.45,86,100,6.38,PK,1692931793
3,3,vorgashor,67.5833,63.9500,6.56,97,100,6.15,RU,1692931794
4,4,tiksi,71.6872,128.8694,12.10,50,100,7.48,RU,1692931797
...,...,...,...,...,...,...,...,...,...,...
564,564,hafar al-batin,28.4328,45.9708,33.32,17,0,0.00,SA,1692932530
565,565,port alfred,-33.5906,26.8910,16.58,52,100,3.21,ZA,1692932532
566,566,natal,-5.7950,-35.2094,25.12,88,40,3.09,BR,1692932329
567,567,colonia,50.9333,6.9500,18.82,89,0,1.54,DE,1692932410


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# %%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.8
)

# Display the map plot
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather = city_data_df.loc[(city_data_df['Max Temp'] >= 24) & \
                                (city_data_df['Max Temp'] < 30) & \
                                (city_data_df['Wind Speed'] < 4.5) & \
                                (city_data_df['Cloudiness'] == 0)
                                ]

# Drop any rows with null values
ideal_weather_df = ideal_weather.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
47,47,metlili chaamba,32.2667,3.6333,29.52,20,0,1.54,DZ,1692931851
57,57,remire-montjoly,4.9167,-52.2667,25.02,94,0,0.00,GF,1692931864
103,103,carloforte,39.1450,8.3058,25.41,65,0,2.62,IT,1692931920
112,112,pullman,46.7313,-117.1796,25.34,31,0,2.06,US,1692931931
209,209,chorbane,35.2858,10.3858,25.44,59,0,1.57,TN,1692932055
233,233,cottage grove,44.8277,-92.9438,26.68,83,0,2.06,US,1692932067
255,255,tucumcari,35.1717,-103.7250,28.86,39,0,3.60,US,1692932121
331,331,mitchell,43.7094,-98.0298,25.99,83,0,3.09,US,1692931953
360,360,santarem,-2.4431,-54.7083,27.07,83,0,2.06,BR,1692932259
389,389,korla,41.7597,86.1469,28.07,23,0,3.85,CN,1692932295


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Create a DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

C:\Users\admin\AppData\Local\Temp\ipykernel_6244\2769619027.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
47,metlili chaamba,DZ,32.2667,3.6333,20,
57,remire-montjoly,GF,4.9167,-52.2667,94,
103,carloforte,IT,39.1450,8.3058,65,
112,pullman,US,46.7313,-117.1796,31,
209,chorbane,TN,35.2858,10.3858,59,
233,cottage grove,US,44.8277,-92.9438,83,
255,tucumcari,US,35.1717,-103.7250,39,
331,mitchell,US,43.7094,-98.0298,83,
360,santarem,BR,-2.4431,-54.7083,83,
389,korla,CN,41.7597,86.1469,23,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000

limit = 20

categories = 'accommodation.hotel'

params = {
    "categories": categories,
    "apiKey": geoapify_key,
    "limit": limit    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
metlili chaamba - nearest hotel: No hotel found
remire-montjoly - nearest hotel: Complexe Belova
carloforte - nearest hotel: Hotel Riviera
pullman - nearest hotel: Quality Inn
chorbane - nearest hotel: No hotel found
cottage grove - nearest hotel: Country Inn & Suites
tucumcari - nearest hotel: Pow Wow Inn
mitchell - nearest hotel: Quality Inn
santarem - nearest hotel: Prático Hotel
korla - nearest hotel: Silver Star Hotel
santa luzia - nearest hotel: Alcoboca
orting - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
al bawiti - nearest hotel: Old Oasis Hotel
brindisi - nearest hotel: Villa Rosmarino
bay harbor - nearest hotel: Comfort Suites
siwa oasis - nearest hotel: فندق الكيلانى
heroica matamoros - nearest hotel: Cameron Hotel
marshall - nearest hotel: Wyndham Garden Lake-Guntersville
yaypan - nearest hotel: No hotel found
zaysan - nearest hotel: Рахат қонақүй


,City,Country,Lat,Lng,Humidity,Hotel Name
47,metlili chaamba,DZ,32.2667,3.6333,20,No hotel found
57,remire-montjoly,GF,4.9167,-52.2667,94,Complexe Belova
103,carloforte,IT,39.1450,8.3058,65,Hotel Riviera
112,pullman,US,46.7313,-117.1796,31,Quality Inn
209,chorbane,TN,35.2858,10.3858,59,No hotel found
233,cottage grove,US,44.8277,-92.9438,83,Country Inn & Suites
255,tucumcari,US,35.1717,-103.7250,39,Pow Wow Inn
331,mitchell,US,43.7094,-98.0298,83,Quality Inn
360,santarem,BR,-2.4431,-54.7083,83,Prático Hotel
389,korla,CN,41.7597,86.1469,23,Silver Star Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
#%%capture --no-display

# Configure the map
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City",
    alpha = 0.8,
    hover_cols = ["Country", "Hotel Name"],
    xlabel = "Longitude",
    ylabel = "Latitude"
)

# Display the map plot
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)